In [1]:
import numpy as np
import h5py

import random

In [2]:
test_X = None
test_Y = None
with h5py.File("./data/ntu/NTU_CS.h5" , 'r') as f:
    test_X = f['test_x'][:]
    test_Y = np.argmax(f['test_y'][:], -1)

In [3]:
import matplotlib
matplotlib.use('Agg')
import os
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, writers
from mpl_toolkits.mplot3d import Axes3D
import glob

In [18]:
def render_animation(poses, filename):
    output = "./test.mp4"
    fps=30
    bitrate = 3000
    limit = 300
    
    parents = [-1, 0, 20, 2, 20, 4, 5, 6, 20, 8, 9, 10, 0, 12, 13, 14, 0, 16, 17, 18, 1, 7, 7, 11, 11]
    
    radius = 1.5
    
    plt.ioff()
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    ax.set_xlim3d([-radius/2, radius/2])
    ax.set_zlim3d([-radius/2, radius/2])
    ax.set_ylim3d([-radius/2, radius/2])
    ax.dist = 12.5
    ax.set_title("3D skeleton") #, pad=35

    lines_3d = []
    trajectories = []

    initialized = False
    image = None
    lines = []
    points = None

    def update_video(i):
        nonlocal initialized, lines, points

        ax.set_xlim3d([-radius/2 + poses[i, 3*2], radius/2 + poses[i, 3*2]])
        ax.set_ylim3d([-radius/2 + poses[i, 3*2+2], radius/2 + poses[i, 3*2+2]])

        if not initialized:
            
            for j, j_parent in enumerate(parents):
                if j_parent == -1:
                    continue

                pos = poses[i]
#                 print(pos)
                print(type(pos))
                lines_3d.append(ax.plot([pos[3*j], pos[3*j_parent]],
                                           [pos[3*j + 2], pos[3*j_parent + 2]],
                                           [pos[3*j + 1], pos[3*j_parent + 1]], zdir='z'))

            #points = ax_in.scatter(*keypoints[i].T, 5, color='red', edgecolors='white', zorder=10)

            initialized = True
        else:

            for j, j_parent in enumerate(parents):
                if j_parent == -1:
                    continue

                pos = poses[i]
#                 print(pos)
                print(type(pos))
                lines_3d[j-1][0].set_xdata([pos[3*j], pos[3*j_parent]])
                lines_3d[j-1][0].set_ydata([pos[3*j + 2], pos[3*j_parent + 2]])
                lines_3d[j-1][0].set_3d_properties([pos[3*j + 1], pos[3*j_parent + 1]], zdir='z')

        
        print('{} {}/{}      '.format(filename, i+1, limit), end='\r')
        

    fig.tight_layout()
    print(f"{filename}")
    anim = FuncAnimation(fig, update_video, frames=np.arange(0, limit), interval=1000/fps, repeat=False)
    if output.endswith('.mp4'):
        Writer = writers['ffmpeg']
        writer = Writer(fps=fps, metadata={}, bitrate=bitrate)
        anim.save(filename, writer=writer)
    elif output.endswith('.gif'):
        anim.save(filename, dpi=80, writer='imagemagick')
    else:
        raise ValueError('Unsupported output format (only .mp4 and .gif are supported)')
    plt.close()


In [11]:
def _rot(rot):
    cos_r, sin_r = rot.cos(), rot.sin()
    zeros = rot.new(rot.size()[:2] + (1,)).zero_()
    ones = rot.new(rot.size()[:2] + (1,)).fill_(1)

    r1 = torch.stack((ones, zeros, zeros),dim=-1)
    rx2 = torch.stack((zeros, cos_r[:,:,0:1], sin_r[:,:,0:1]), dim = -1)
    rx3 = torch.stack((zeros, -sin_r[:,:,0:1], cos_r[:,:,0:1]), dim = -1)
    rx = torch.cat((r1, rx2, rx3), dim = 2)

    ry1 = torch.stack((cos_r[:,:,1:2], zeros, -sin_r[:,:,1:2]), dim =-1)
    r2 = torch.stack((zeros, ones, zeros),dim=-1)
    ry3 = torch.stack((sin_r[:,:,1:2], zeros, cos_r[:,:,1:2]), dim =-1)
    ry = torch.cat((ry1, r2, ry3), dim = 2)

    rz1 = torch.stack((cos_r[:,:,2:3], sin_r[:,:,2:3], zeros), dim =-1)
    r3 = torch.stack((zeros, zeros, ones),dim=-1)
    rz2 = torch.stack((-sin_r[:,:,2:3], cos_r[:,:,2:3],zeros), dim =-1)
    rz = torch.cat((rz1, rz2, r3), dim = 2)

    rot = rz.matmul(ry).matmul(rx)
    return rot

In [12]:
def transform(x, transform_type):
    if transform_type == "reverse":
        return np.flip(x, 0)
    if transform_type == "rotation":
        theta = 5
        print(x.shape)
        x = x.reshape(x.shape[0], 2, x.shape[1]//2)
        x = x.reshape(x.shape[0], x.shape[1], x.shape[2]//3, 3)
        print(x.shape)
        rot = np.random.randint(theta, size=(1, 3))*0.1
        print(rot.shape)
        print(rot)

In [13]:
origin = test_X[2]
reverse = transform(origin, "reverse")
rotation = transform(origin, "rotation")

(300, 150)
(300, 2, 25, 3)
(1, 3)
[[0.1 0.2 0.2]]


In [19]:
render_animation(origin, "./origin.mp4")
render_animation(reverse, "./reverse.mp4")

./origin.mp4
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


AttributeError: 'list' object has no attribute 'shape'

array([[-0.02281147, -0.256312  ,  0.06750298, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02270117, -0.2566558 ,  0.06731105, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02267188, -0.2572357 ,  0.06716204, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)